In [1]:
import os

In [2]:
%pwd

'c:\\Users\\USER\\DataScience\\Projects\\End-To-End-Mushroom-with-MLFLOW\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'c:\\Users\\USER\\DataScience\\Projects\\End-To-End-Mushroom-with-MLFLOW'

## Entity setup

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    min_samples_split: int
    target_column: str